<a href="https://colab.research.google.com/github/kitti-tao/Realistic-Talking-Head-Model/blob/master/Realistic_Talking_head_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Prevent random disconnects**

This cell runs JS code to automatic reconnect to runtime.

In [6]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
  function ClickConnect(){
    console.log("Working"); 
    document.querySelector("colab-connect-button").click() 
  }
  var connect_timer = setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


**Check GPU**

* Google Colab can provide you with one of Tesla graphics cards: K80, T4, P4 or P100
* Here you can check the model of GPU before using Realistic-Talking-Head-model

In [7]:
!nvidia-smi

Fri Mar 27 10:36:38 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

**Clone Github repository and install requirements**
*   Clone Github repository or pull updates
*   Requirements install is automatically
*   Automatically sets timer to prevent random disconnects





In [7]:
#@title Clone or pull DeepFaceLab from Github

Mode = "clone" #@param ["clone", "pull"]

from pathlib import Path
if (Mode == "clone"):
  !git clone git@github.com:kitti-tao/Realistic-Talking-Head-Model.git

  # fix linux warning
  # /usr/lib/python3.6/multiprocessing/semaphore_tracker.py:143: UserWarning: semaphore_tracker: There appear to be 1 leaked semaphores to clean up at shutdown
  fin = open("/usr/lib/python3.6/multiprocessing/semaphore_tracker.py", "rt")
  data = fin.read()
  data = data.replace('if cache:', 'if False:')
  fin.close()

  fin = open("/usr/lib/python3.6/multiprocessing/semaphore_tracker.py", "wt")
  fin.write(data)
  fin.close()
else:
  %cd /content/Realistic-Neural-Talking-Head-Models
  !git pull


Cloning into 'Realistic-Talking-Head-Model'...
Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [0]:
!pip install -r /content/Realistic-Neural-Talking-Head-Models/requirements.txt
!apt-get install cuda-10-0
!pip install gdown

if not Path("/content/vgg_face").exists():
  print("Downloading vgg_face model ... ")
  !mkdir /content/vgg_face
  !wget -q --no-check-certificate -r http://www.robots.ox.ac.uk/~vgg/software/vgg_face/src/vgg_face_caffe.tar.gz -O /content/vgg_face/vgg_face_caffe.tar.gz
  !tar -C /content/vgg_face/ xvzf /content/vgg_face/vgg_face_caffe.tar.gz
  !sudo apt install caffe-cuda
  !pip install mmdnn
  !pip install numpy==1.16.1
  !mmtoir -f caffe -n /content/vgg_face/vgg_face_caffe/VGG_FACE_deploy.prototxt -w /content/vgg_face/vgg_face_caffe/VGG_FACE.caffemodel -o /content/vgg_face/VGGFACE_IR
  !mmtocode -f pytorch -n /content/vgg_face/VGGFACE_IR.pb --IRWeightPath /content/vgg_face/VGGFACE_IR.npy --dstModelPath /content/vgg_face/Pytorch_VGGFACE_IR.py -dw /content/vgg_face/Pytorch_VGGFACE_IR.npy
  !mmtomodel -f pytorch -in /content/vgg_face/Pytorch_VGGFACE_IR.py -iw /content/vgg_face/Pytorch_VGGFACE_IR.npy -o /content/vgg_face/Pytorch_VGGFACE.pth
  !mv /content/vgg_face/Pytorch_VGGFACE_IR.py /content/Realistic-Neural-Talking-Head-Models/.
  !mv /content/vgg_face/Pytorch_VGGFACE.pth /content/Realistic-Neural-Talking-Head-Models/.
  !rm -r /content/vgg_face
  !pip install numpy

import IPython
from google.colab import output

display(IPython.display.Javascript('''
  function ClickConnect(){
    console.log("Working"); 
    document.querySelector("colab-connect-button").click() 
  }
  var connect_timer = setInterval(ClickConnect,60000)
'''))

print("\nDone!")

Connect to Google Drive

In [1]:
#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

def zip_and_copy(path, mode):
  unzip_cmd=" -q "+Archive_name
  
  %cd $path
  copy_cmd = "/content/drive/My\ Drive/"+Archive_name+" "+path
  !cp $copy_cmd
  !unzip $unzip_cmd    
  !rm $Archive_name

if Mode == "workspace":
  zip_and_copy("/content", "workspace")
elif Mode == "models":
  zip_and_copy("/content/workspace", "model")
  
print("Done!")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content
Done!


Download Dataset

In [0]:
!wget --user voxceleb1912 --password 0s42xuw6: -q --no-check-certificate -r http://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox1a/vox2_test_mp4.zip -O /content/vox2_test_mp4.zip
!unzip -q /content/vox2_test_mp4.zip -d /content/Realistic-Neural-Talking-Head-Models/
!rm /content/vox2_test_mp4.zip

Train Model

In [0]:
!python /content/Realistic-Neural-Talking-Head-Models/train.py